In [25]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np 
import pandas as pd
import random 
from tqdm.auto import tqdm 
from pathlib import Path

In [26]:

df = pd.read_csv('train.csv')
modes = df.mode().iloc[0]
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Cabin'] = df['Cabin'].apply(lambda x: str(x)[0] if not pd.isnull(x) else 'NoCabin')
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode().iloc[0])
df['LogFare'] = np.log1p(df['Fare'])
df['LogAge'] =  np.log1p(df['Age'])
df = pd.get_dummies(df,columns=["Sex","Pclass","Embarked","Cabin"])
df['SibSp_Parch'] = df['SibSp'] + df['Parch']
df = df.drop(['Parch','PassengerId','SibSp'],axis=1)
df = df.select_dtypes(np.number)
dfS = torch.tensor(df['Survived'].values,dtype=torch.long)
df = df.drop(['Survived','Cabin_T'],axis = 1)
columnsA = df.columns
corr = np.where(df.corr("pearson").iloc[1] > 0.)
columnsA = list(columnsA[corr])
print(type(columnsA))
dfData = torch.tensor(df[columnsA].values,dtype = torch.float)
print(dfS,dfData)

<class 'list'>
tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 0,

In [27]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.dererministic = True
same_seeds(63)

In [28]:
class TrainDataset(Dataset):
    def __init__(self,data,label,mode):
        self.data = data
        self.label = label
        self.mode = mode
        
    def __getitem__(self,idx):
        if self.mode == 'train' or self.mode=='valid':
            return self.data[idx] , self.label[idx]
        else:
            return self.data[idx]
        
    def __len__(self):
            return len(self.data)

In [29]:
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel,self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(dfData.shape[1],512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0,3),
            nn.Linear(512,64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0,3),
            nn.Linear(64,32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0,3),
            nn.Linear(32,2),
        )
    def forward(self,x):
        output = self.fc_layer(x)
        return output

In [30]:
BATCH_SIZE = 8
def split_dataset():
    train_idx = random.sample(range(0,dfData.shape[0] - 1),730)
    valid_idx = [idx for idx in range(dfData.shape[0]) if idx not in train_idx]
    print('Train Sample num = ', len(train_idx))
    print('Valid Sample num = ', len(valid_idx))
    train_data = dfData[train_idx]
    train_label= dfS[train_idx]
    print('Train data shape = ', train_data.shape )
    valid_data = dfData[valid_idx]
    valid_label = dfS[valid_idx]
    print('Valid Data Shape = ', valid_data.shape )
    train_dataset = TrainDataset(train_data , train_label , mode='train')
    valid_dataset = TrainDataset(valid_data , valid_label , mode='valid')
    
    train_loader  = DataLoader(  train_dataset,batch_size=BATCH_SIZE , shuffle = True , drop_last = True)
    valid_loader  = DataLoader(  valid_dataset,batch_size=BATCH_SIZE)
    return train_loader,valid_loader
train_loader , valid_loader = split_dataset()



Train Sample num =  730
Valid Sample num =  161
Train data shape =  torch.Size([730, 13])
Valid Data Shape =  torch.Size([161, 13])


In [31]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
early_stop = 50

EPOCHS = 500
LR = 0.0003
loss_fn = nn.CrossEntropyLoss()
model_paths = []
os.makedirs('model', exist_ok=True)

model = LinearModel().to(device)
optimizer = torch.optim.AdamW(model.parameters() , lr = LR)
model.train()

stop_count = 0
best_acc = 0.0

model_path  = f'model/linear.ckpt'
model_paths.append(model_path)
train_loader , valid_loader = split_dataset()
for epoch in range(EPOCHS):
    model.train()
    total = 0
    correct = 0
    train_loss = []
    for idx,batch in enumerate(tqdm(train_loader)):
        datas,labels = batch
        datas  = datas.to(device)
        labels = labels.to(device)
        logits = model(datas)
        loss = loss_fn(logits,labels)
        preds = torch.argmax(logits,dim=-1)
        total += labels.shape[0]
        correct += (preds==labels).sum().item()
        loss.backward()
        train_loss.append(loss.item())

        optimizer.step()
        optimizer.zero_grad()
    train_acc = correct /total
    train_loss = sum(train_loss) / len(train_loss)
    print(f'[Epochs = {epoch}] Train Accuracy = {train_acc:.5f},Train Loss = {train_loss:.5f}')


    model.eval()

    total = 0
    correct = 0
    valid_loss = []
    for idx,batch in enumerate(tqdm(valid_loader)):
        datas,labels = batch
        datas  = datas.to(device)
        labels = labels.to(device)
        logits = model(datas)
        loss = loss_fn(logits,labels)
        preds = torch.argmax(logits,dim=-1)
        total += labels.shape[0]
        correct += (preds==labels).sum().item()
        loss.backward()
        valid_loss.append(loss.item())

        optimizer.step()
        optimizer.zero_grad()
    valid_acc = correct /total
    vaild_loss = sum(valid_loss) / len(valid_loss)
    print(f'[Epochs = {epoch}]  vaild Accuracy = {valid_acc:.5f},vaild Loss = {vaild_loss:.5f}')
    
    
    
    if valid_acc >= best_acc:
        best_acc = valid_acc
        torch.save(model.state_dict(), model_path)
        print(f'Save model with accuracy = {best_acc:.5f}')
        stop_count = 0
    else:
        stop_count +=1
        if stop_count > early_stop:
            print('Early Stop!!')
            break

df = pd.read_csv('test.csv')
#print(df)
df['Cabin'] = df['Cabin'].apply(lambda x: str(x)[0] if not pd.isnull(x) else 'NoCabin')
df['LogFare'] = np.log1p(df['Fare'])
df['LogAge'] =  np.log1p(df['Age'])
df = pd.get_dummies(df,columns=["Sex","Pclass","Embarked","Cabin"])
df['SibSp_Parch'] = df['SibSp'] + df['Parch']
df = df.drop('Parch',axis=1)
df = df.drop('SibSp',axis=1)
df = df.select_dtypes(np.number)
dfID = df['PassengerId'].values
df = df.drop(['PassengerId'],axis=1)
dfData = torch.tensor(df[columnsA].values,dtype = torch.float)

test_dataset = TrainDataset(dfData , label=None,mode =  'test')
test_loader = DataLoader(test_dataset , batch_size = BATCH_SIZE) 
model1 = LinearModel().to(device)
model1.load_state_dict(torch.load(model_paths[0]))
model1.eval()
models = [model1]
pred_result = []
for idx,batch in enumerate(tqdm(test_loader)):
    datas = batch
    datas = datas.to(device)
    for idx,model in enumerate(models):
        if idx ==0: logits = model(datas)
        else: logits += model(datas)
    preds = torch.argmax(logits,dim=-1)
    pred_result +=list(preds)
    
with open('pred_test.csv','w') as f:
    f.write('PassengerId,Survived\n')
    for idx,pred in zip(dfID , pred_result):
        f.write(f'{idx},{pred}\n')


Train Sample num =  730
Valid Sample num =  161
Train data shape =  torch.Size([730, 13])
Valid Data Shape =  torch.Size([161, 13])


  0%|          | 0/91 [00:00<?, ?it/s]

[Epochs = 0] Train Accuracy = 0.63599,Train Loss = 0.64081


  0%|          | 0/21 [00:00<?, ?it/s]

[Epochs = 0]  vaild Accuracy = 0.71429,vaild Loss = 0.58988
Save model with accuracy = 0.71429


...

  0%|          | 0/91 [00:00<?, ?it/s]

[Epochs = 233] Train Accuracy = 0.79670,Train Loss = 0.45726


  0%|          | 0/21 [00:00<?, ?it/s]

[Epochs = 233]  vaild Accuracy = 0.83851,vaild Loss = 0.38241
Early Stop!!


  0%|          | 0/53 [00:00<?, ?it/s]